In [88]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.collocations import *
import pymorphy2

In [89]:
morph = pymorphy2.MorphAnalyzer()

In [90]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

загружаю весь корпус размеченных отзывов

In [91]:
f = open('SentiRuEval_rest_markup_train.xml', 'r', encoding='utf-8')
a = f.read()
review_all = a.split('###___###')
# print(review_all[1])

здесь я создаю массив нормализованных слов для дальнейшего создания коллокаций + убрала знаки препинания

In [92]:
words = []
for review in review_all:
    text = re.findall('<text>(.*?)</text>', review)
    for i in text:
        i = re.sub('[.,!?():;]', ' ', i)
        for word in word_tokenize(i.lower()):
            words.append(morph.parse(word)[0].normal_form)

# Сначала построю коллокации-биграммы

убрала стоп-слова, удалила всякие маленькие слова

если делать окно, то результаты выдачи десяти лучших коллокаций хуже

In [93]:
# finder = BigramCollocationFinder.from_words(words, window_size = 20)
finder = BigramCollocationFinder.from_words(words)
stopwords = nltk.corpus.stopwords.words('russian')
stopwords.append('очень')
stopwords.append('спасибо')
finder.apply_word_filter(lambda w: (len(w) < 3) or (w.lower() in stopwords))

In [95]:
finder.nbest(bigram_measures.likelihood_ratio, 300)

[('день', 'рождение'),
 ('молодая', 'человек'),
 ('это', 'заведение'),
 ('живой', 'музыка'),
 ('остаться', 'довольный'),
 ('обслуживающий', 'персонал'),
 ('это', 'ресторан'),
 ('добрый', 'день'),
 ('высокий', 'уровень'),
 ('приятно', 'удивить'),
 ('обязательно', 'прийти'),
 ('мочь', 'сказать'),
 ('свой', 'дело'),
 ('данный', 'заведение'),
 ('хотеть', 'отметить'),
 ('вкусный', 'еда'),
 ('достаточно', 'быстро'),
 ('красивый', 'интерьер'),
 ('конец', 'вечер'),
 ('всякий', 'похвала'),
 ('заказать', 'столик'),
 ('прийти', 'ещё'),
 ('салат', 'цезарь'),
 ('будний', 'день'),
 ('первое', 'этаж'),
 ('детский', 'меню'),
 ('оставлять', 'желать'),
 ('общий', 'впечатление'),
 ('провести', 'время'),
 ('посетить', 'данный'),
 ('честно', 'говорить'),
 ('люба', 'вкус'),
 ('самый', 'главный'),
 ('милый', 'девушка'),
 ('долго', 'ждать'),
 ('второе', 'этаж'),
 ('помочь', 'определиться'),
 ('приветливый', 'персонал'),
 ('куриный', 'печень'),
 ('выше', 'всякий'),
 ('детский', 'комната'),
 ('барна', 'стойка')

теперь сохраню отранжированный список коллокаций с их значением. метрику я взяла likelihood_ratio, потому что её выдача среди 10-ти лучших коллокаций мне понравилась больше всего

In [96]:
scored_bigrams = finder.score_ngrams(bigram_measures.student_t)
rest_col = scored_bigrams
rest_col

[(('это', 'заведение'), 5.794126438499476),
 (('день', 'рождение'), 5.639311322324996),
 (('это', 'ресторан'), 5.57858270373927),
 (('остаться', 'довольный'), 4.3219801882806035),
 (('молодая', 'человек'), 4.227140912111958),
 (('живой', 'музыка'), 3.86430082488129),
 (('мочь', 'сказать'), 3.558296887203882),
 (('вкусный', 'еда'), 3.5253535318795652),
 (('заказать', 'столик'), 3.509363724624007),
 (('добрый', 'день'), 3.449777875086122),
 (('это', 'место'), 3.4314470116428155),
 (('обслуживающий', 'персонал'), 3.3072969098700318),
 (('данный', 'заведение'), 3.281341068519443),
 (('прийти', 'ещё'), 3.26038713409579),
 (('высокий', 'уровень'), 3.1544794862941035),
 (('приятно', 'удивить'), 3.154302255069688),
 (('обязательно', 'прийти'), 3.1479928234805015),
 (('хотеть', 'отметить'), 3.1369335950769837),
 (('красивый', 'интерьер'), 3.1248936872317006),
 (('свой', 'дело'), 2.9858018233447914),
 (('достаточно', 'быстро'), 2.9773327355153687),
 (('общий', 'впечатление'), 2.9656752852089876)

открываю колин файл с оценочными словами

In [105]:
t = open('implicit.txt', 'r', encoding='utf-8')
b = t.read()
implis = b.split('\n')

смотрю, в каких коллокациях есть выделенные нами оценочные слова

In [104]:
w1 = open('bi_final_col.txt', 'a')

In [106]:
for col in rest_col:
    col_tag = str(morph.parse(col[0][0])[0].tag.POS) + ' ' + str(morph.parse(col[0][1])[0].tag.POS)
    if int(col[1]) > 0.1:
        for impl in implis:
            impl = impl.split('\t')
            if str(impl[0]) == str(col[0][0]) or str(impl[0]) == str(col[0][1]):
                if col_tag not in 'ADJF ADJF, NOUN NOUN, INFN INFN, INFN ADJF':
                    w1.write(str(col[0][0]) + ' ' + str(col[0][1]) + '; ' + str(impl[2]) + '\n')
                    print(col, col[1], impl[0], impl[2], col_tag)

(('вкусный', 'еда'), 3.5253535318795652) 3.5253535318795652 вкусный pos ADJF NOUN
(('добрый', 'день'), 3.449777875086122) 3.449777875086122 добрый pos ADJF NOUN
(('прийти', 'ещё'), 3.26038713409579) 3.26038713409579 прийти neg INFN ADVB
(('приятно', 'удивить'), 3.154302255069688) 3.154302255069688 приятно pos ADVB INFN
(('обязательно', 'прийти'), 3.1479928234805015) 3.1479928234805015 прийти neg ADVB INFN
(('красивый', 'интерьер'), 3.1248936872317006) 3.1248936872317006 красивый pos ADJF NOUN
(('достаточно', 'быстро'), 2.9773327355153687) 2.9773327355153687 быстро pos ADVB ADVB
(('долго', 'ждать'), 2.7923637277831186) 2.7923637277831186 ждать neg ADVB INFN
(('долго', 'ждать'), 2.7923637277831186) 2.7923637277831186 долго neg ADVB INFN
(('приветливый', 'персонал'), 2.619441797792427) 2.619441797792427 приветливый pos ADJF NOUN
(('ждать', 'минута'), 2.575070090491762) 2.575070090491762 ждать neg INFN NOUN
(('весь', 'понравиться'), 2.5593370347073403) 2.5593370347073403 понравиться pos AD

PROFIT

но это, конечно, ещё не всё. я попробую сделать то же, но для триграмм. потому что потому

# Теперь я построю коллокации-триграммы

In [107]:
# finder = TrigramCollocationFinder.from_words(words, window_size = 20)
tr_finder = TrigramCollocationFinder.from_words(words)
stopwords = nltk.corpus.stopwords.words('russian')
stopwords.append('спасибо')
tr_finder.apply_word_filter(lambda w: (len(w) < 3 and w != 'не') or (w.lower() in stopwords and w != 'не'))

In [108]:
tr_finder.nbest(trigram_measures.student_t, 300)

[('остаться', 'очень', 'довольный'),
 ('весь', 'очень', 'понравиться'),
 ('отмечать', 'день', 'рождение'),
 ('свой', 'день', 'рождение'),
 ('понравиться', 'абсолютно', 'весь'),
 ('не', 'мочь', 'не'),
 ('обязательно', 'прийти', 'ещё'),
 ('большой', 'не', 'пойти'),
 ('весь', 'очень', 'вкусно'),
 ('выше', 'всякий', 'похвала'),
 ('оставлять', 'желать', 'хороший'),
 ('столик', 'заказать', 'заранее'),
 ('посетить', 'это', 'замечательный'),
 ('весь', 'остаться', 'довольный'),
 ('оказаться', 'очень', 'вкусный'),
 ('не', 'мочь', 'сказать'),
 ('профессионал', 'свой', 'дело'),
 ('принести', 'довольно', 'быстро'),
 ('прошлый', 'ещё', 'минута'),
 ('блюдо', 'понравиться', 'абсолютно'),
 ('посетить', 'данный', 'ресторан'),
 ('весь', 'добрый', 'день'),
 ('никто', 'не', 'встретить'),
 ('не', 'прийтись', 'ждать'),
 ('очень', 'красивый', 'интерьер'),
 ('очень', 'приятный', 'впечатление'),
 ('сказать', 'не', 'мочь'),
 ('очень', 'понравиться', 'салат'),
 ('очень', 'вкусный', 'блюдо'),
 ('очень', 'вкусно', 

In [109]:
scored_trigrams = tr_finder.score_ngrams(trigram_measures.student_t)
tr_rest_col = scored_trigrams
tr_rest_col

[(('остаться', 'очень', 'довольный'), 3.6049042328578924),
 (('весь', 'очень', 'понравиться'), 3.308290686484289),
 (('отмечать', 'день', 'рождение'), 2.4494261658325964),
 (('свой', 'день', 'рождение'), 2.449398918568061),
 (('понравиться', 'абсолютно', 'весь'), 2.449111656211476),
 (('не', 'мочь', 'не'), 2.4326858609561057),
 (('обязательно', 'прийти', 'ещё'), 2.2359713614613845),
 (('большой', 'не', 'пойти'), 2.23520886255681),
 (('весь', 'очень', 'вкусно'), 2.23057398918688),
 (('выше', 'всякий', 'похвала'), 1.9999995113858926),
 (('оставлять', 'желать', 'хороший'), 1.9999907526290064),
 (('столик', 'заказать', 'заранее'), 1.9998380244233789),
 (('посетить', 'это', 'замечательный'), 1.9997206942157761),
 (('весь', 'остаться', 'довольный'), 1.9988635910811654),
 (('оказаться', 'очень', 'вкусный'), 1.998819173266717),
 (('не', 'мочь', 'сказать'), 1.9977687366981092),
 (('профессионал', 'свой', 'дело'), 1.7320471321895108),
 (('принести', 'довольно', 'быстро'), 1.7319329504038563),
 (

In [110]:
w2 = open('tri_final_col.txt', 'a')

In [111]:
for col in tr_rest_col:
    col_tag = str(morph.parse(col[0][0])[0].tag.POS) + ' ' + str(morph.parse(col[0][1])[0].tag.POS) + ' ' + str(morph.parse(col[0][2])[0].tag.POS)
    if int(col[1]) > 0.05:
        for impl in implis:
            impl = impl.split('\t')
            if str(impl[0]) == str(col[0][0]) or str(impl[0]) == str(col[0][1]) or str(impl[0]) == str(col[0][2]):
                w2.write(str(col[0][0]) + ' ' + str(col[0][1]) + ' ' + str(col[0][2]) + '; ' + str(impl[2]) + '\n')
                print(col, impl[0], impl[2], col_tag)

(('весь', 'очень', 'понравиться'), 3.308290686484289) понравиться pos ADJF ADVB INFN
(('понравиться', 'абсолютно', 'весь'), 2.449111656211476) понравиться pos INFN ADVB ADJF
(('обязательно', 'прийти', 'ещё'), 2.2359713614613845) прийти neg ADVB INFN ADVB
(('весь', 'очень', 'вкусно'), 2.23057398918688) вкусно pos ADJF ADVB ADVB
(('оставлять', 'желать', 'хороший'), 1.9999907526290064) хороший pos INFN INFN ADJF
(('столик', 'заказать', 'заранее'), 1.9998380244233789) заранее pos NOUN INFN ADVB
(('посетить', 'это', 'замечательный'), 1.9997206942157761) замечательный pos INFN PRCL ADJF
(('оказаться', 'очень', 'вкусный'), 1.998819173266717) вкусный pos INFN ADVB ADJF
(('профессионал', 'свой', 'дело'), 1.7320471321895108) профессионал pos NOUN ADJF NOUN
(('принести', 'довольно', 'быстро'), 1.7319329504038563) быстро pos INFN ADVB ADVB
(('блюдо', 'понравиться', 'абсолютно'), 1.7318513577879229) понравиться pos NOUN INFN ADVB
(('весь', 'добрый', 'день'), 1.7316462063329248) добрый pos ADJF ADJF

done